In [ ]:
import sys
import numpy as np
import cv2


In [ ]:
filename = './googlenet/fig/apple2.png'

img = cv2.imread(filename)

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Network load fialed')
    sys.exit()
    
# 분류명 불러오기
classNames = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')
# scalefactor 
# deepleaning 그림 이미지 사이즈는 (244, 244) 사이즈로미리 결정하고, 고정시킨다.
# 이미지 설정은 config 파일을 찾아서 확인할 수 밖에 없다.
blob = cv2.dnn.blobFromImage(img, 1, (244, 244), (104, 117, 123), swapRB = False)
net.setInput(blob)
prob = net.forward()

print(prob.shape)


In [ ]:
# prob는 다루기 쉽게 자른다.
out = prob.flatten()
classId = np.argmax(out)

print(classId)
print(classNames[classId])
print(out[classId])

In [ ]:
confidence = out[classId]
category = classNames[classId]
text = f'{category} ({confidence*100:4.2f} %)'
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
            1, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 슬라이드로 보여주기

import sys
import numpy as np
import cv2
import glob

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Network load fialed')
    sys.exit()
    
# 분류명 불러오기
classNames = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')

img_lists = glob.glob('./googlenet/fig/sweet/image*.jpg')

cv2.namedWindow('scene', cv2.WINDOW_NORMAL)
cv2.resizeWindow('scene', 488, 488) 

idx = 0
while True:
    if len(img_lists) <= 0:
        print('images is zeros')
        break
    
    print(idx)
    img = cv2.imread(img_lists[idx])
    if img is None:
        print('image read failed')
        break
        
        
    blob = cv2.dnn.blobFromImage(img, 1, (244, 244), (104, 117, 123), swapRB = False)
    net.setInput(blob)
    prob = net.forward()
    out = prob.flatten()
    classId = np.argmax(out)
    confidence = out[classId]
    category = classNames[classId]
    text = f'{category} ({confidence*100:4.2f} %)'
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
                0.5, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.imshow('scene', img)


    if cv2.waitKey(3000) == 27:
        break

    idx +=1
    if idx >= len(img_lists):
        idx = 0
            


cv2.destroyAllWindows()


In [ ]:
# face detection network download 
# 얼굴 인식
import numpy as np
import sys
import cv2

img = cv2.imread('./opencv_face_detector/fig/sunglass.png')

model = './opencv_face_detector/opencv_face_detector_uint8.pb'
config = './opencv_face_detector/opencv_face_detector.pbtxt'

face_detect_net = cv2.dnn.readNet(model, config)

if face_detect_net.empty():
    print('net load error')
    sys.exit
    
blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                     swapRB = False)

face_detect_net.setInput(blob)

out = face_detect_net.forward()

h, w = img.shape[:2]

# out은 1, 1 200 * 7 배열이 나온다. 
# 해당 배열은 0, 1번째는 사용하지 않고, 3번째 부터 사용한다.
# 2번째는 사물이 얼굴로 분류된 확률이다.
# 3, 4, 5, 6열은 (x1, y1), (x2, y2) 좌표이며, 사각형을 만들 수 있는 좌표다.
# 값이 확률로 기록되어 확인을 해서 계산을 해줘야 한다.
# 앞에 1, 1은 사용하지 않으므로 0으로 제거해준다.
detect = out[0, 0, :, :]
for i in range(detect.shape[0]):
    confidence = detect[i, 2] 
    
    if confidence  > 0.1:  # 얼굴 인식할 box 위치를 뽑아낸다.
        x1 = int(detect[i, 3] * w) 
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w) 
        y2 = int(detect[i, 6] * h) 
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        text = 'Face: {}%'.format(round(confidence* 100, 2))
        cv2.putText(img, text, (x1, y1 - 4), cv2.FONT_HERSHEY_COMPLEX,
                    0.5, (0, 0, 255), 1, cv2.LINE_AA)
        
print(out.shape)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import sys
import numpy as np
import cv2

#model = './opencv_face_detector/res10_300x300_ssd_iter_140000_fp16.caffemode'
model = './opencv_face_detector/opencv_face_detector_uint8.pb'
config = './opencv_face_detector/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('video open failed')
    sys.exit()
    
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('net load failed')
    sys.exit()
    
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('frame read failed')
        break
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()
    
    detect = out[0, 0, :, :]
    h, w = frame.shape[:2]
    
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        
        if confidence > 0.5:
            x1 = int(detect[i, 3] * w) 
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w) 
            y2 = int(detect[i, 6] * h) 

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = 'Face: {}%'.format(round(confidence* 100, 2))
            cv2.putText(frame, text, (x1, y1 - 4), cv2.FONT_HERSHEY_COMPLEX,
                        0.5, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    if cv2.waitKey(30) == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

In [13]:
from tensorflow import keras

In [14]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape)
print(x_train.dtype)

# 영상으로 쓰기 위해서 차원을 1로 스케일링 한다. + float 값으로 변환
x_train = x_train.reshape(60000, 28, 28, 1)/255.
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)/255.

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
# one hot encding
#y_train = keras.utils.to_categorical(y_train)

print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

(60000, 28, 28)
uint8
float64
float64
float32
float32


In [15]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size  = (3, 3), input_shape= (28, 28, 1), 
                              activation = 'relu'))
model.add(keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPool2D(pool_size = 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10, activation = 'softmax'))

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 9216)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               1179776   
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                      

In [16]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = 'accuracy')
modelpath = './mnist_mymodel/{epoch:003d}-{val_loss:.4f}.h5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath = modelpath,
                                        save_best_only = True)
early_stopping = keras.callbacks.EarlyStopping(patience = 10)
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(x_train, y_train, epochs = 10, batch_size = 200,
                    verbose = 1, validation_split=0.3, 
                    callbacks = [checkpoint, early_stopping])


# history = model.fit(x_train, y_train, validation_data = (x_test, y_test),
#                     epochs = 30, batch_size = 200, verbose = 1,
#                     callbacks = [early_stopping, checkpointer])

Epoch 1/10
210/210 [==============================] - 69s 328ms/step - loss: 0.2857 - accuracy: 0.9135 - val_loss: 0.0759 - val_accuracy: 0.9770
Epoch 2/10
210/210 [==============================] - 65s 309ms/step - loss: 0.0727 - accuracy: 0.9782 - val_loss: 0.0537 - val_accuracy: 0.9844
Epoch 3/10
210/210 [==============================] - 65s 308ms/step - loss: 0.0520 - accuracy: 0.9838 - val_loss: 0.0523 - val_accuracy: 0.9837
Epoch 4/10
210/210 [==============================] - 65s 311ms/step - loss: 0.0363 - accuracy: 0.9880 - val_loss: 0.0471 - val_accuracy: 0.9862
Epoch 5/10
210/210 [==============================] - 68s 325ms/step - loss: 0.0294 - accuracy: 0.9907 - val_loss: 0.0430 - val_accuracy: 0.9874
Epoch 6/10
210/210 [==============================] - 66s 316ms/step - loss: 0.0236 - accuracy: 0.9921 - val_loss: 0.0466 - val_accuracy: 0.9873
Epoch 7/10
210/210 [==============================] - 66s 313ms/step - loss: 0.0198 - accuracy: 0.9930 - val_loss: 0.0593 - val_ac

In [17]:
print('\n Test accuracy: {:.4f}%'.format(model.evaluate(x_test, y_test)[1]))
print(model.evaluate(x_test, y_test))

313/313 [==============================] - 3s 10ms/step - loss: 0.0325 - accuracy: 0.9909

 Test accuracy: 0.9909%
313/313 [==============================] - 3s 10ms/step - loss: 0.0325 - accuracy: 0.9909
[0.03245822340250015, 0.9908999800682068]


In [18]:
model.save('./model_mnist/', include_optimizer=False)

INFO:tensorflow:Assets written to: ./model_mnist/assets


In [19]:
# 오닉스 세이브를 위한 설치
!pip install -U tf2onnx

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


In [20]:
!python -m tf2onnx.convert --saved-model model_mnist --output model_mnist.onnx

# 설치하고 나서 model_mnist.onnx를 이동한다.

2022-05-04 15:59:55.867680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-04 15:59:55.867724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
C:\Users\sangt\anaconda3\envs\tensorflow\lib\runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-04 15:59:58.980653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-05-04 15:59:58.980695: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-04 15:59:58.984086: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA dia

In [9]:

import sys
import numpy as np
import cv2

def on_mouse(event, x, y, flags, param):
    global oldx, oldy
    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), 255, 10, cv2.LINE_AA)
            cv2.imshow('image', img)
            oldx, oldy = x, y

# 무게 중심 함수
def norm_digit(img):
    m = cv2.moments(img)
    cx = m['m10'] / m['m00']
    cy = m['m01'] / m['m00']
    h,w = img.shape[:2]
    aff = np.array([[1,0, w/2-cx], [0,1,h/2-cy]], dtype = np.float32)
    dst = cv2.warpAffine(img, aff, (0,0))
    return dst


net = cv2.dnn.readNet('./model_mnist.onnx')

if net.empty():
    print('Net load failed')
    sys.exit()
    
img = np.zeros((400, 400), np.uint8)
cv2.imshow('image', img)
cv2.setMouseCallback('image', on_mouse)



while True:
    key = cv2.waitKey()
    
    if key == 27:
        break
        
    elif key == ord(' '):
        
        #직접 그린 숫자 400*400짜리 이미지를 blob이라는 객체로 변환
        #blob은 4차원 형식의 ndarray (N,C,H,W)
        #학습 시, 학습데이터 픽셀값을 0~1까지 정규화하여 학습했으므로 1/255.을 넣어서 똑같이 정규화 해준다
        #학습 데이터 mnist 크기그 28*28이였으므로 똑같은 크기인 28*28 resize 시행 
        #내가 그린 숫자를 가운데에 위치시켜 놓고 진행: norm_digit
        blob = cv2.dnn.blobFromImage(norm_digit(img), 1/255, (28, 28))
        net.setInput(blob)
        prob = net.forward()
        
        '''
        prob: (1,10) 10개의 elemnet가 있고,
        확률이 최대값인 element의 인덱스값 = 모델이 인식한 숫자
        maxLoc : 가장 확률이 높은 결과
        maxVal : 확률
        '''
        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        
        digit = maxLoc[0]
        
        print(f'{digit} ({maxVal*100:4.2f}%)')
        img.fill(0)
        cv2.imshow('image', img)
        
cv2.destroyAllWindows()

9 (96.90%)
6 (88.28%)
9 (66.83%)
1 (15.90%)
2 (69.08%)
3 (96.06%)
4 (64.04%)
5 (88.65%)


In [ ]:
moveWindow